<a href="https://colab.research.google.com/github/pragneshrana/WorkSample/blob/main/LLM/VectorDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain
!pip install pinecone-client
!pip install pypdf
!pip install openai
!pip install tiktoken


## Pinecone

In [2]:
from pinecone import Pinecone

pc = Pinecone(api_key="9b60cfda-9598-4050-90e6-a628119f4aea")
index = pc.Index("testing")

In [3]:
index.upsert(
    vectors=[
        {
            "id": "vec1",
            "values": [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1],
            "metadata": {"genre": "drama"}
        }, {
            "id": "vec2",
            "values": [0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2],
            "metadata": {"genre": "action"}
        }, {
            "id": "vec3",
            "values": [0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3],
            "metadata": {"genre": "drama"}
        }, {
            "id": "vec4",
            "values": [0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4],
            "metadata": {"genre": "action"}
        }
    ],
    namespace= "ns1"
)

{'upserted_count': 4}

In [4]:
index.query(
    namespace="ns1",
    vector=[0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3],
    top_k=2,
    include_values=True,
    include_metadata=True,
    filter={"genre": {"$eq": "action"}}
)

{'matches': [{'id': 'vec2',
              'metadata': {'genre': 'action'},
              'score': 1.0,
              'values': [0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2]},
             {'id': 'vec4',
              'metadata': {'genre': 'action'},
              'score': 1.0,
              'values': [0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4]}],
 'namespace': 'ns1',
 'usage': {'read_units': 6}}

## ChromaDB

In [5]:
%pip install -Uq chromadb numpy datasets

In [10]:
#### Loading the data into Chroma
# https://www.datacamp.com/tutorial/chromadb-tutorial-step-by-step-guide
# Chroma comes with a built-in embedding model, which makes it simple to load text.

# Import Chroma and instantiate a client. The default Chroma client is ephemeral, meaning it will not save to disk.
import chromadb
from chromadb.config import Settings


client = chromadb.PersistentClient(path="db/")

In [11]:
# Create a new Chroma collection to store the supporting evidence, don't need to specify an embedding fuction, and the default will be used.
collection = client.create_collection(name="Students")



In [13]:
#data
student_info = """
Alexandra Thompson, a 19-year-old computer science sophomore with a 3.7 GPA,
is a member of the programming and chess clubs who enjoys pizza, swimming, and hiking
in her free time in hopes of working at a tech company after graduating from the University of Washington.
"""

club_info = """
The university chess club provides an outlet for students to come together and enjoy playing
the classic strategy game of chess. Members of all skill levels are welcome, from beginners learning
the rules to experienced tournament players. The club typically meets a few times per week to play casual games,
participate in tournaments, analyze famous chess matches, and improve members' skills.
"""

university_info = """
The University of Washington, founded in 1861 in Seattle, is a public research university
with over 45,000 students across three campuses in Seattle, Tacoma, and Bothell.
As the flagship institution of the six public universities in Washington state,
UW encompasses over 500 buildings and 20 million square feet of space,
including one of the largest library systems in the world.
"""

In [14]:
collection.add(
    documents = [student_info, club_info, university_info],
    metadatas = [{"source": "student info"},{"source": "club info"},{'source':'university info'}],
    ids = ["id1", "id2", "id3"]
)

In [16]:
## Quering
results = collection.query(
    query_texts=["What is the student name?"],
    n_results=2
)

results

{'ids': [['id1', 'id2']],
 'distances': [[1.2946666564424738, 1.3954030668049473]],
 'metadatas': [[{'source': 'student info'}, {'source': 'club info'}]],
 'embeddings': None,
 'documents': [['\nAlexandra Thompson, a 19-year-old computer science sophomore with a 3.7 GPA,\nis a member of the programming and chess clubs who enjoys pizza, swimming, and hiking\nin her free time in hopes of working at a tech company after graduating from the University of Washington.\n',
   "\nThe university chess club provides an outlet for students to come together and enjoy playing\nthe classic strategy game of chess. Members of all skill levels are welcome, from beginners learning\nthe rules to experienced tournament players. The club typically meets a few times per week to play casual games,\nparticipate in tournaments, analyze famous chess matches, and improve members' skills.\n"]],
 'uris': None,
 'data': None}

In [18]:
## For deployment wth docker container
https://abhishektatachar.medium.com/run-chroma-db-on-a-local-machine-and-as-a-docker-container-a9d4b91d2a97